In [2]:
from pymongo import MongoClient
from datetime import datetime, timedelta
from bson.objectid import ObjectId
from faker import Faker 
import random
import pprint as pp

In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client["Employee"]
collection = db["employee_db"]

fake = Faker()

In [15]:
# ... Add more sample employees with similar structure but different values

# Let's add more employees programmatically
# Initialize Faker
fake = Faker()

# Define data pools for realistic variety
departments = [
    "Engineering", "Marketing", "Sales", "HR", "Finance", 
    "Operations", "Product", "Design", "Legal", "Customer Support"
]

positions = {
    "Engineering": ["Junior Developer", "Senior Developer", "Lead Developer", "Engineering Manager", "DevOps Engineer", "QA Engineer", "Data Engineer"],
    "Marketing": ["Marketing Specialist", "Content Manager", "SEO Specialist", "Brand Manager", "Marketing Director", "Digital Marketing Manager"],
    "Sales": ["Sales Representative", "Account Executive", "Sales Manager", "Business Development", "Regional Sales Director"],
    "HR": ["HR Specialist", "Recruiter", "HR Manager", "People Operations", "Training Coordinator"],
    "Finance": ["Financial Analyst", "Accountant", "Finance Manager", "Controller", "CFO", "Budget Analyst"],
    "Operations": ["Operations Manager", "Process Analyst", "Supply Chain Manager", "Operations Director"],
    "Product": ["Product Manager", "Product Owner", "Product Analyst", "VP of Product", "UX Researcher"],
    "Design": ["UX Designer", "UI Designer", "Graphic Designer", "Design Lead", "Creative Director"],
    "Legal": ["Legal Counsel", "Paralegal", "Compliance Officer", "Legal Assistant"],
    "Customer Support": ["Support Specialist", "Customer Success Manager", "Technical Support", "Support Lead"]
}

skill_pools = {
    "Engineering": ["Python", "JavaScript", "Java", "React", "Node.js", "MongoDB", "PostgreSQL", "Docker", "Kubernetes", "AWS", "Git", "Linux"],
    "Marketing": ["Google Analytics", "SEO", "Content Marketing", "Social Media", "Email Marketing", "Adobe Creative Suite", "HubSpot"],
    "Sales": ["Salesforce", "CRM", "Lead Generation", "Negotiation", "B2B Sales", "Cold Calling", "Account Management"],
    "HR": ["HRIS", "Talent Acquisition", "Employee Relations", "Compensation", "Benefits Administration", "Training"],
    "Finance": ["Excel", "QuickBooks", "Financial Analysis", "Budgeting", "SAP", "Tableau", "SQL"],
    "Operations": ["Project Management", "Lean Six Sigma", "Supply Chain", "Process Improvement", "ERP"],
    "Product": ["Product Strategy", "Roadmapping", "Agile", "Scrum", "User Research", "Analytics", "Wireframing"],
    "Design": ["Adobe Creative Suite", "Figma", "Sketch", "Prototyping", "User Research", "HTML/CSS"],
    "Legal": ["Contract Law", "Compliance", "Legal Research", "Litigation", "Corporate Law"],
    "Customer Support": ["Zendesk", "Customer Service", "Technical Troubleshooting", "CRM", "Communication"]
}

us_states = ["CA", "NY", "TX", "FL", "IL", "PA", "OH", "GA", "NC", "MI", "NJ", "VA", "WA", "AZ", "MA", "TN", "IN", "MO", "MD", "WI"]

def generate_employee_record(emp_id):
    """Generate a single employee record"""
    
    # Basic info
    first_name = fake.first_name()
    last_name = fake.last_name()
    department = random.choice(departments)
    position = random.choice(positions[department])
    
    # Experience and salary based on position level
    if "Junior" in position or "Specialist" in position:
        years_exp = random.randint(0, 3)
        salary_base = random.randint(45000, 70000)
    elif "Senior" in position or "Lead" in position or "Manager" in position:
        years_exp = random.randint(3, 8)
        salary_base = random.randint(70000, 120000)
    elif "Director" in position or "VP" in position or "CFO" in position:
        years_exp = random.randint(8, 20)
        salary_base = random.randint(120000, 200000)
    else:
        years_exp = random.randint(1, 12)
        salary_base = random.randint(55000, 95000)
    
    # Add some salary variation
    salary = salary_base + random.randint(-10000, 15000)
    salary = max(salary, 35000)  # Minimum salary floor
    
    # Performance rating (normally distributed around 3.5)
    performance = round(max(1.0, min(5.0, random.normalvariate(3.5, 0.8))), 1)
    
    # Skills (3-6 skills from department pool)
    dept_skills = skill_pools[department]
    num_skills = random.randint(3, 6)
    skills = random.sample(dept_skills, min(num_skills, len(dept_skills)))
    
    # Dates
    hire_date = fake.date_between(start_date='-10y', end_date='today')
    
    # Last promotion (if they've been there long enough)
    if (datetime.now().date() - hire_date).days > 365:
        promotion_start = hire_date + timedelta(days=365)
        last_promotion = fake.date_between(start_date=promotion_start, end_date='today')
    else:
        last_promotion = None
    
    # Address
    state = random.choice(us_states)
    city = fake.city()
    zip_code = fake.zipcode()
    
    # Remote work (higher chance for certain departments)
    remote_likely_depts = ["Engineering", "Design", "Marketing", "Product"]
    is_remote = random.choices([True, False], 
                              weights=[0.7, 0.3] if department in remote_likely_depts else [0.3, 0.7])[0]
    
    return {
        "_id": str(ObjectId()),
        "employee_id": f"EMP{emp_id:05d}",
        "first_name": first_name,
        "last_name": last_name,
        "email": f"{first_name.lower()}.{last_name.lower()}@company.com",
        "department": department,
        "position": position,
        "salary": salary,
        "years_experience": years_exp,
        "performance_rating": performance,
        "skills": skills,
        "hire_date": hire_date.isoformat(),
        "last_promotion": last_promotion.isoformat() if last_promotion else None,
        "is_remote": is_remote,
        "address": {
            "city": city,
            "state": state,
            "zip_code": zip_code
        }
    }

def generate_employee_database(num_employees=10000):
    """Generate the complete employee database"""
    print(f"Generating {num_employees} employee records...")
    
    employees = []
    for i in range(1, num_employees + 1):
        if i % 1000 == 0:
            print(f"Generated {i} employees...")
        
        employee = generate_employee_record(i)
        employees.append(employee)
    
    print(f"✅ Successfully generated {len(employees)} employee records!")
    return employees

# Generate the database
employee_db = generate_employee_database(10000)  # Changed variable name from employee to employee_db

# Display some sample records
print("\n" + "="*50)
print("SAMPLE RECORDS")
print("="*50)
for i in range(3):
    print(f"\nEmployee {i+1}:")
    for key, value in employee_db[i].items():
        print(f"  {key}: {value}")

# Convert to DataFrame for easy analysis
df = pd.json_normalize(employee_db)
print(f"\n" + "="*50)
print("DATABASE SUMMARY")
print("="*50)
print(f"Total employees: {len(df)}")
print(f"Departments: {df['department'].value_counts().to_dict()}")
print(f"Average salary: ${df['salary'].mean():,.2f}")
print(f"Remote workers: {df['is_remote'].sum()} ({df['is_remote'].mean()*100:.1f}%)")

# Optional: Save to different formats
print(f"\n" + "="*50)
print("EXPORT OPTIONS")
print("="*50)
print("To save your data, run any of these commands:")
print("# Save as JSON")
print("with open('employees.json', 'w') as f:")
print("    json.dump(employee_db, f, indent=2, default=str)")
print()
print("# Save as CSV")
print("df.to_csv('employees.csv', index=False)")
print()
print("# Save as Excel")
print("df.to_excel('employees.xlsx', index=False)")

# MongoDB insertion example
print(f"\n" + "="*50)
print("MONGODB INSERTION EXAMPLE")
print("="*50)
print("""
# To insert into MongoDB:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['employee']
collection = db['employee_db']

# Insert all records
result = collection.insert_many(employee_db)
print(f"Inserted {len(result.inserted_ids)} employees into MongoDB")
""")

Generating 10000 employee records...
Generated 1000 employees...
Generated 2000 employees...
Generated 3000 employees...
Generated 4000 employees...
Generated 5000 employees...
Generated 6000 employees...
Generated 7000 employees...
Generated 8000 employees...
Generated 9000 employees...
Generated 10000 employees...
✅ Successfully generated 10000 employee records!

SAMPLE RECORDS

Employee 1:
  _id: 689d04b1bcde0633f7de6ae2
  employee_id: EMP00001
  first_name: Deborah
  last_name: Moses
  email: deborah.moses@company.com
  department: Legal
  position: Paralegal
  salary: 94141
  years_experience: 4
  performance_rating: 2.8
  skills: ['Litigation', 'Corporate Law', 'Contract Law', 'Legal Research']
  hire_date: 2024-02-15
  last_promotion: 2025-05-22
  is_remote: False
  address: {'city': 'Sarahville', 'state': 'NC', 'zip_code': '58886'}

Employee 2:
  _id: 689d04b1bcde0633f7de6ae3
  employee_id: EMP00002
  first_name: Stephanie
  last_name: Moore
  email: stephanie.moore@company.com

In [14]:
import pandas as pd

In [16]:
df.to_csv('employees.csv', index=False)

In [20]:
# Save as Excel
df.to_excel('employees.xlsx', index=False)


In [21]:
# To insert into MongoDB:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['employee']
collection = db['employee_db']

# Insert all records
result = collection.insert_many(employee_db)
print(f"Inserted {len(result.inserted_ids)} employees into MongoDB")

Inserted 10000 employees into MongoDB


In [23]:
# =============================================================================
# QUERY 1: High Performers Query
# Find all employees with performance rating >= 4.0 AND salary > 80,000
# Return only their name, department, salary, and performance rating
# =============================================================================

print("\n1. HIGH PERFORMERS QUERY")
print("-" * 40)

query1 = {
    "$and": [
        {"performance_rating": {"$gte": 4.0}},
        {"salary": {"$gt": 80000}}
    ]
}

projection1 = {
    "_id": 0,
    "first_name": 1,
    "last_name": 1,
    "department": 1,
    "salary": 1,
    "performance_rating": 1
}

print("MongoDB Query:")
print(f"db.employees.find({query1}, {projection1})")
print()

# Execute query
result1 = list(collection.find(query1, projection1))
print(f"Found {len(result1)} high performers")

if result1:
    print("\nSample Results:")
    df1 = pd.DataFrame(result1)
    print(df1.head(10).to_string(index=False))
    print(f"\nAverage salary of high performers: ${df1['salary'].mean():,.2f}")
    print(f"Average rating: {df1['performance_rating'].mean():.2f}")


1. HIGH PERFORMERS QUERY
----------------------------------------
MongoDB Query:
db.employees.find({'$and': [{'performance_rating': {'$gte': 4.0}}, {'salary': {'$gt': 80000}}]}, {'_id': 0, 'first_name': 1, 'last_name': 1, 'department': 1, 'salary': 1, 'performance_rating': 1})

Found 1660 high performers

Sample Results:
first_name last_name       department  salary  performance_rating
     Bruce  Gonzalez            Sales  170889                 5.0
      Lori    Wright          Finance   95119                 5.0
      Mark     Gomez       Operations  176603                 4.4
 Catherine   Collins      Engineering   97276                 4.4
   Brianna  Thompson            Sales  179777                 4.0
      Jodi    Peters Customer Support   95752                 4.1
     Bobby     Garza       Operations  100551                 4.0
    Brenda     Kelly        Marketing  118450                 4.5
 Catherine      Long               HR   87630                 4.7
   Stephen     G

In [33]:
# QUERY 2: Experience-Based Filtering
# Find employees with 5-10 years of experience (inclusive) who earn between $70,000 and $120,000
# Project only essential contact information (name, email, department)
# =============================================================================

print("\n\n2. EXPERIENCE-BASED FILTERING")
print("-" * 40)

query2 = {
    "$and": [
        {"years_experience": {"$gte": 5, "$lte": 10}},
        {"salary": {"$gte": 70000, "$lte": 120000}}
    ]
}

projection2 = {
    "_id": 0,
    "first_name": 1,
    "last_name": 1,
    "email": 1,
    "department": 1
}

print("MongoDB Query:")
print(f"db.employees.find({query2}, {projection2})")
print()

# Execute query
result2 = list(collection.find(query2, projection2))
print(f"Found {len(result2)} employees matching experience and salary criteria")

if result2:
    print("\nSample Results:")
    df2 = pd.DataFrame(result2)
    print(df2.head(10).to_string(index=False))
    print(f"\nDepartment distribution:")
    print(df2['department'].value_counts().head())



2. EXPERIENCE-BASED FILTERING
----------------------------------------
MongoDB Query:
db.employees.find({'$and': [{'years_experience': {'$gte': 5, '$lte': 10}}, {'salary': {'$gte': 70000, '$lte': 120000}}]}, {'_id': 0, 'first_name': 1, 'last_name': 1, 'email': 1, 'department': 1})

Found 3450 employees matching experience and salary criteria

Sample Results:
first_name last_name                      email       department
    Sherry      Wood    sherry.wood@company.com           Design
    George      Chen    george.chen@company.com               HR
    Teresa      Neal    teresa.neal@company.com          Finance
     James    Fisher   james.fisher@company.com               HR
   Crystal    Monroe crystal.monroe@company.com            Legal
      Mary  Valencia  mary.valencia@company.com       Operations
       Joe    Valdez     joe.valdez@company.com          Product
     Maria     Davis    maria.davis@company.com          Finance
     Jason    Wilson   jason.wilson@company.com     

In [32]:
# QUERY 3: Salary Range Analysis
# Find employees whose salary is NOT in the range of $60,000-$100,000
# Showing their full name (concatenated), current salary, and years of experience
# =============================================================================

print("\n\n3. SALARY RANGE ANALYSIS")
print("-" * 40)

query3 = {
    "$or": [
        {"salary": {"$lt": 60000}},
        {"salary": {"$gt": 100000}}
    ]
}

# Using aggregation pipeline for concatenated full name
pipeline3 = [
    {"$match": query3},
    {"$project": {
        "_id": 0,
        "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
        "salary": 1,
        "years_experience": 1
    }}
]

print("MongoDB Aggregation Pipeline:")
print(f"db.employees.aggregate({pipeline3})")
print()

# Execute aggregation
result3 = list(collection.aggregate(pipeline3))
print(f"Found {len(result3)} employees outside $60K-$100K salary range")

if result3:
    print("\nSample Results:")
    df3 = pd.DataFrame(result3)
    print(df3.head(10).to_string(index=False))
    
    # Analysis
    high_earners = len([emp for emp in result3 if emp['salary'] > 100000])
    low_earners = len([emp for emp in result3 if emp['salary'] < 60000])
    print(f"\nBreakdown:")
    print(f"- High earners (>$100K): {high_earners}")
    print(f"- Lower earners (<$60K): {low_earners}")



3. SALARY RANGE ANALYSIS
----------------------------------------
MongoDB Aggregation Pipeline:
db.employees.aggregate([{'$match': {'$or': [{'salary': {'$lt': 60000}}, {'salary': {'$gt': 100000}}]}}, {'$project': {'_id': 0, 'full_name': {'$concat': ['$first_name', ' ', '$last_name']}, 'salary': 1, 'years_experience': 1}}])

Found 3732 employees outside $60K-$100K salary range

Sample Results:
 salary  years_experience          full_name
 127334                 7       Ronald Scott
 170889                14     Bruce Gonzalez
 116760                 7        George Chen
 176603                13         Mark Gomez
 156767                16    Laurie Williams
 179777                19   Brianna Thompson
 107073                 3       Damon Grimes
  45938                 1 Cassandra Mcintosh
  57651                 2          Jay Horne
  57195                 0       Eric Cochran

Breakdown:
- High earners (>$100K): 2715
- Lower earners (<$60K): 1017


In [31]:
# QUERY 4: Recent Hires
# Find employees hired in the last 2 years with performance rating > 3.5
# Return custom fields showing "full_name", "tenure_months", and "annual_salary"
# =============================================================================

from datetime import datetime, timedelta
import pandas as pd
from pymongo import MongoClient

print("\n\n4. RECENT HIRES")
print("-" * 40)

# Calculate date 2 years ago
two_years_ago = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')

# Aggregation pipeline for complex calculations
pipeline4 = [
    {"$match": {
        "$and": [
            {"hire_date": {"$gte": two_years_ago}},
            {"performance_rating": {"$gt": 3.5}}
        ]
    }},
    {"$addFields": {
        "hire_date_obj": {"$dateFromString": {"dateString": "$hire_date"}}
    }},
    {"$project": {
        "_id": 0,
        "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
        "tenure_months": {
            "$divide": [
                {"$subtract": ["$$NOW", "$hire_date_obj"]},  # Changed new Date() to $$NOW
                1000 * 60 * 60 * 24 * 30.44  # Convert milliseconds to months
            ]
        },
        "annual_salary": "$salary",
        "performance_rating": 1,
        "department": 1,
        "hire_date": 1
    }},
    {"$addFields": {
        "tenure_months": {"$round": ["$tenure_months", 1]}
    }},
    {"$sort": {"hire_date": -1}}  # Most recent hires first
]

print("MongoDB Aggregation Pipeline:")
print(f"db.employees.aggregate({pipeline4})")
print()

# Execute aggregation
result4 = list(collection.aggregate(pipeline4))
print(f"Found {len(result4)} recent high-performing hires")

if result4:
    print("\nSample Results:")
    df4 = pd.DataFrame(result4)
    display_cols = ['full_name', 'tenure_months', 'annual_salary', 'performance_rating', 'department']
    print(df4[display_cols].head(10).to_string(index=False))
    
    print(f"\nAnalysis of recent hires:")
    print(f"- Average tenure: {df4['tenure_months'].mean():.1f} months")
    print(f"- Average salary: ${df4['annual_salary'].mean():,.2f}")
    print(f"- Average rating: {df4['performance_rating'].mean():.2f}")




4. RECENT HIRES
----------------------------------------
MongoDB Aggregation Pipeline:
db.employees.aggregate([{'$match': {'$and': [{'hire_date': {'$gte': '2023-08-14'}}, {'performance_rating': {'$gt': 3.5}}]}}, {'$addFields': {'hire_date_obj': {'$dateFromString': {'dateString': '$hire_date'}}}}, {'$project': {'_id': 0, 'full_name': {'$concat': ['$first_name', ' ', '$last_name']}, 'tenure_months': {'$divide': [{'$subtract': ['$$NOW', '$hire_date_obj']}, 2630016000.0]}, 'annual_salary': '$salary', 'performance_rating': 1, 'department': 1, 'hire_date': 1}}, {'$addFields': {'tenure_months': {'$round': ['$tenure_months', 1]}}}, {'$sort': {'hire_date': -1}}])

Found 916 recent high-performing hires

Sample Results:
       full_name  tenure_months  annual_salary  performance_rating       department
    David Gibson            0.1          83294                 3.6            Legal
  Jacob Cummings            0.1         114304                 4.7          Product
   Crystal Jones          

In [34]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure, PyMongoError
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Dict, Any, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EmployeeQueryManager:
    """
    A class to handle MongoDB employee database queries with proper error handling
    and comprehensive result analysis.
    """
    
    def __init__(self, connection_string: str = 'mongodb://localhost:27017/', 
                 db_name: str = 'company_db', collection_name: str = 'employees'):
        """
        Initialize the query manager with database connection details.
        
        Args:
            connection_string: MongoDB connection string
            db_name: Database name
            collection_name: Collection name
        """
        self.connection_string = connection_string
        self.db_name = db_name
        self.collection_name = collection_name
        self.client = None
        self.db = None
        self.collection = None
        
    def connect(self) -> bool:
        """
        Establish connection to MongoDB database.
        
        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            self.client = MongoClient(self.connection_string, serverSelectionTimeoutMS=5000)
            # Test the connection
            self.client.admin.command('ping')
            self.db = self.client[self.db_name]
            self.collection = self.db[self.collection_name]
            logger.info(f"Successfully connected to MongoDB: {self.db_name}.{self.collection_name}")
            return True
        except ConnectionFailure as e:
            logger.error(f"Failed to connect to MongoDB: {e}")
            return False
        except Exception as e:
            logger.error(f"Unexpected error during connection: {e}")
            return False
    
    def close_connection(self):
        """Close the MongoDB connection."""
        if self.client:
            self.client.close()
            logger.info("MongoDB connection closed")

    def find_high_performers(self, min_rating: float = 4.0, min_salary: int = 80000) -> Dict[str, Any]:
        """
        Query 1: Find all employees with performance rating >= 4.0 AND salary > 80,000.
        Return only their name, department, salary, and performance rating.
        
        Operator Choices Explained:
        - $and: Explicitly combines multiple conditions (though implicit AND would work)
        - $gte: Greater than or equal to for inclusive rating threshold
        - $gt: Greater than for exclusive salary threshold (as specified)
        
        Args:
            min_rating: Minimum performance rating (default 4.0)
            min_salary: Minimum salary threshold (default 80000)
            
        Returns:
            Dict containing query results and metadata
        """
        try:
            # Query using $and operator for explicit multiple conditions
            # $gte for inclusive rating, $gt for exclusive salary threshold
            query = {
                "$and": [
                    {"performance_rating": {"$gte": min_rating}},  # Inclusive: rating >= 4.0
                    {"salary": {"$gt": min_salary}}                # Exclusive: salary > 80000
                ]
            }
            
            # Projection to return only required fields
            projection = {
                "_id": 0,              # Exclude MongoDB ObjectId
                "first_name": 1,       # Include first name
                "last_name": 1,        # Include last name  
                "department": 1,       # Include department
                "salary": 1,           # Include salary
                "performance_rating": 1 # Include performance rating
            }
            
            logger.info(f"Executing high performers query: rating >= {min_rating}, salary > {min_salary}")
            
            # Execute query with error handling
            cursor = self.collection.find(query, projection)
            results = list(cursor)
            
            # Calculate statistics
            if results:
                df = pd.DataFrame(results)
                stats = {
                    "avg_salary": df['salary'].mean(),
                    "avg_rating": df['performance_rating'].mean(),
                    "salary_range": (df['salary'].min(), df['salary'].max()),
                    "dept_distribution": df['department'].value_counts().to_dict()
                }
            else:
                stats = {}
            
            return {
                "success": True,
                "query": query,
                "projection": projection,
                "total_count": len(results),
                "results": results,
                "statistics": stats,
                "sample_results": results[:5]  # First 5 for display
            }
            
        except OperationFailure as e:
            logger.error(f"Database operation failed in high_performers query: {e}")
            return {"success": False, "error": str(e), "results": []}
        except Exception as e:
            logger.error(f"Unexpected error in high_performers query: {e}")
            return {"success": False, "error": str(e), "results": []}

    def find_experienced_mid_earners(self, min_exp: int = 5, max_exp: int = 10, 
                                   min_salary: int = 70000, max_salary: int = 120000) -> Dict[str, Any]:
        """
        Query 2: Find employees with 5-10 years of experience (inclusive) who earn between $70,000 and $120,000.
        Project only essential contact information (name, email, department).
        
        Operator Choices Explained:
        - $and: Combines multiple range conditions
        - $gte & $lte: Used for inclusive ranges (5-10 years inclusive, $70K-$120K inclusive)
        - Single field projection: More efficient than excluding unwanted fields
        
        Args:
            min_exp: Minimum years of experience (default 5)
            max_exp: Maximum years of experience (default 10)
            min_salary: Minimum salary (default 70000)
            max_salary: Maximum salary (default 120000)
            
        Returns:
            Dict containing query results and metadata
        """
        try:
            # Query using range operators for inclusive boundaries
            # $gte and $lte create inclusive ranges for both experience and salary
            query = {
                "$and": [
                    {"years_experience": {"$gte": min_exp, "$lte": max_exp}},  # 5-10 years inclusive
                    {"salary": {"$gte": min_salary, "$lte": max_salary}}       # $70K-$120K inclusive
                ]
            }
            
            # Project only essential contact information
            projection = {
                "_id": 0,
                "first_name": 1,
                "last_name": 1, 
                "email": 1,
                "department": 1
            }
            
            logger.info(f"Executing experienced mid-earners query: {min_exp}-{max_exp} years, ${min_salary}-${max_salary}")
            
            cursor = self.collection.find(query, projection)
            results = list(cursor)
            
            # Get additional stats by running separate query
            if results:
                # Get full records for statistics
                stats_cursor = self.collection.find(query)
                full_results = list(stats_cursor)
                df = pd.DataFrame(full_results)
                stats = {
                    "avg_experience": df['years_experience'].mean(),
                    "avg_salary": df['salary'].mean(),
                    "dept_distribution": df['department'].value_counts().to_dict(),
                    "experience_range": (df['years_experience'].min(), df['years_experience'].max())
                }
            else:
                stats = {}
            
            return {
                "success": True,
                "query": query,
                "projection": projection,
                "total_count": len(results),
                "results": results,
                "statistics": stats,
                "sample_results": results[:5]
            }
            
        except OperationFailure as e:
            logger.error(f"Database operation failed in experienced_mid_earners query: {e}")
            return {"success": False, "error": str(e), "results": []}
        except Exception as e:
            logger.error(f"Unexpected error in experienced_mid_earners query: {e}")
            return {"success": False, "error": str(e), "results": []}

    def find_salary_outliers(self, exclude_min: int = 60000, exclude_max: int = 100000) -> Dict[str, Any]:
        """
        Query 3: Find employees whose salary is NOT in the range of $60,000-$100,000.
        Show their full name (concatenated), current salary, and years of experience.
        
        Operator Choices Explained:
        - $or: Logical OR to find salaries outside the range (below min OR above max)
        - $lt: Less than for salaries below the range
        - $gt: Greater than for salaries above the range
        - $concat: Aggregation operator to concatenate first and last names
        - Alternative: Could use $nor with single range condition, but $or is more explicit
        
        Args:
            exclude_min: Lower bound of salary range to exclude (default 60000)
            exclude_max: Upper bound of salary range to exclude (default 100000)
            
        Returns:
            Dict containing query results and metadata
        """
        try:
            # Using aggregation pipeline for field concatenation and complex logic
            # $or operator finds records outside the range (salary < 60K OR salary > 100K)
            pipeline = [
                {
                    "$match": {
                        "$or": [
                            {"salary": {"$lt": exclude_min}},  # Below range
                            {"salary": {"$gt": exclude_max}}   # Above range
                        ]
                    }
                },
                {
                    "$project": {
                        "_id": 0,
                        # $concat operator combines first_name + space + last_name
                        "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
                        "salary": 1,
                        "years_experience": 1
                    }
                },
                {
                    "$sort": {"salary": -1}  # Sort by salary descending for easier analysis
                }
            ]
            
            logger.info(f"Executing salary outliers query: excluding ${exclude_min}-${exclude_max} range")
            
            cursor = self.collection.aggregate(pipeline)
            results = list(cursor)
            
            # Analyze high vs low earners
            if results:
                high_earners = [emp for emp in results if emp['salary'] > exclude_max]
                low_earners = [emp for emp in results if emp['salary'] < exclude_min]
                
                stats = {
                    "high_earners_count": len(high_earners),
                    "low_earners_count": len(low_earners),
                    "avg_salary_high": sum(emp['salary'] for emp in high_earners) / len(high_earners) if high_earners else 0,
                    "avg_salary_low": sum(emp['salary'] for emp in low_earners) / len(low_earners) if low_earners else 0,
                    "salary_distribution": {
                        "highest": max(emp['salary'] for emp in results),
                        "lowest": min(emp['salary'] for emp in results)
                    }
                }
            else:
                stats = {}
            
            return {
                "success": True,
                "pipeline": pipeline,
                "total_count": len(results),
                "results": results,
                "statistics": stats,
                "sample_results": results[:5]
            }
            
        except OperationFailure as e:
            logger.error(f"Database operation failed in salary_outliers query: {e}")
            return {"success": False, "error": str(e), "results": []}
        except Exception as e:
            logger.error(f"Unexpected error in salary_outliers query: {e}")
            return {"success": False, "error": str(e), "results": []}

    def find_recent_high_performers(self, years_back: int = 2, min_rating: float = 3.5) -> Dict[str, Any]:
        """
        Query 4: Find employees hired in the last 2 years with performance rating > 3.5.
        Return custom fields showing "full_name", "tenure_months", and "annual_salary".
        
        Operator Choices Explained:
        - $gte: Greater than or equal for date comparison (hire_date >= cutoff_date)
        - $gt: Greater than for exclusive rating threshold
        - $dateFromString: Converts ISO date strings to MongoDB date objects for calculations
        - $subtract & $divide: Mathematical operators for tenure calculation
        - $concat: String concatenation for full name
        - $round: Rounds tenure to 1 decimal place for readability
        
        Args:
            years_back: How many years back to look for recent hires (default 2)
            min_rating: Minimum performance rating threshold (default 3.5)
            
        Returns:
            Dict containing query results and metadata
        """
        try:
            # Calculate cutoff date (2 years ago)
            cutoff_date = (datetime.now() - timedelta(days=years_back * 365)).strftime('%Y-%m-%d')
            
            # Complex aggregation pipeline for date calculations and field transformations
            pipeline = [
                {
                    "$match": {
                        "$and": [
                            {"hire_date": {"$gte": cutoff_date}},     # Recent hires (>= cutoff)
                            {"performance_rating": {"$gt": min_rating}}  # High performers (> 3.5)
                        ]
                    }
                },
                {
                    "$addFields": {
                        # Convert string date to MongoDB date object for calculations
                        "hire_date_obj": {"$dateFromString": {"dateString": "$hire_date"}}
                    }
                },
                {
                    "$project": {
                        "_id": 0,
                        # Concatenate full name as requested
                        "full_name": {"$concat": ["$first_name", " ", "$last_name"]},
                        # Calculate tenure in months using date arithmetic
                        "tenure_months": {
                            "$divide": [
                                {"$subtract": [{"$now": {}}, "$hire_date_obj"]},
                                1000 * 60 * 60 * 24 * 30.44  # Convert milliseconds to months
                            ]
                        },
                        "annual_salary": "$salary",  # Rename for clarity
                        "performance_rating": 1,
                        "department": 1,
                        "hire_date": 1
                    }
                },
                {
                    "$addFields": {
                        # Round tenure to 1 decimal place for readability
                        "tenure_months": {"$round": ["$tenure_months", 1]}
                    }
                },
                {
                    "$sort": {"hire_date": -1}  # Sort by most recent hires first
                }
            ]
            
            logger.info(f"Executing recent high performers query: hired since {cutoff_date}, rating > {min_rating}")
            
            cursor = self.collection.aggregate(pipeline)
            results = list(cursor)
            
            # Calculate statistics
            if results:
                stats = {
                    "avg_tenure_months": sum(emp['tenure_months'] for emp in results) / len(results),
                    "avg_salary": sum(emp['annual_salary'] for emp in results) / len(results),
                    "avg_rating": sum(emp['performance_rating'] for emp in results) / len(results),
                    "dept_distribution": {},
                    "tenure_range": (
                        min(emp['tenure_months'] for emp in results),
                        max(emp['tenure_months'] for emp in results)
                    )
                }
                
                # Department distribution
                dept_counts = {}
                for emp in results:
                    dept = emp['department']
                    dept_counts[dept] = dept_counts.get(dept, 0) + 1
                stats["dept_distribution"] = dept_counts
            else:
                stats = {}
            
            return {
                "success": True,
                "pipeline": pipeline,
                "cutoff_date": cutoff_date,
                "total_count": len(results),
                "results": results,
                "statistics": stats,
                "sample_results": results[:5]
            }
            
        except OperationFailure as e:
            logger.error(f"Database operation failed in recent_high_performers query: {e}")
            return {"success": False, "error": str(e), "results": []}
        except Exception as e:
            logger.error(f"Unexpected error in recent_high_performers query: {e}")
            return {"success": False, "error": str(e), "results": []}

    def run_all_queries(self) -> Dict[str, Any]:
        """
        Execute all four queries and return comprehensive results.
        
        Returns:
            Dict containing all query results with error handling
        """
        if not self.connect():
            return {"success": False, "error": "Failed to connect to database"}
        
        try:
            logger.info("Executing all employee queries...")
            
            results = {
                "query_1_high_performers": self.find_high_performers(),
                "query_2_experienced_mid_earners": self.find_experienced_mid_earners(),
                "query_3_salary_outliers": self.find_salary_outliers(),
                "query_4_recent_high_performers": self.find_recent_high_performers()
            }
            
            # Summary statistics
            total_results = sum(
                query_result.get("total_count", 0) 
                for query_result in results.values() 
                if query_result.get("success", False)
            )
            
            results["summary"] = {
                "total_employees_found": total_results,
                "successful_queries": sum(1 for r in results.values() if r.get("success", False)),
                "timestamp": datetime.now().isoformat()
            }
            
            return results
            
        finally:
            self.close_connection()

def display_query_results(query_result: Dict[str, Any], query_name: str):
    """
    Display formatted query results with sample data.
    
    Args:
        query_result: Result dictionary from query function
        query_name: Name of the query for display purposes
    """
    print("=" * 80)
    print(f"QUERY RESULTS: {query_name}")
    print("=" * 80)
    
    if not query_result.get("success", False):
        print(f"❌ Query failed: {query_result.get('error', 'Unknown error')}")
        return
    
    print(f"✅ Query successful! Found {query_result['total_count']} results")
    
    # Display query/pipeline
    if "query" in query_result:
        print(f"\n🔍 MongoDB Query:")
        print(f"   {query_result['query']}")
        if "projection" in query_result:
            print(f"   Projection: {query_result['projection']}")
    elif "pipeline" in query_result:
        print(f"\n🔍 Aggregation Pipeline:")
        for i, stage in enumerate(query_result['pipeline']):
            print(f"   Stage {i+1}: {stage}")
    
    # Display sample results
    if query_result['sample_results']:
        print(f"\n📊 Sample Results (showing first 3):")
        for i, result in enumerate(query_result['sample_results'][:3], 1):
            print(f"\n   Result {i}:")
            for key, value in result.items():
                if isinstance(value, float):
                    print(f"     {key}: {value:.2f}")
                elif isinstance(value, int) and key in ['salary', 'annual_salary']:
                    print(f"     {key}: ${value:,}")
                else:
                    print(f"     {key}: {value}")
    
    # Display statistics
    if query_result.get('statistics'):
        print(f"\n📈 Statistics:")
        stats = query_result['statistics']
        for key, value in stats.items():
            if isinstance(value, float):
                if 'salary' in key:
                    print(f"   {key}: ${value:,.2f}")
                else:
                    print(f"   {key}: {value:.2f}")
            elif isinstance(value, dict) and key == 'dept_distribution':
                print(f"   {key}: {dict(list(value.items())[:3])}...")  # Show top 3
            else:
                print(f"   {key}: {value}")
    
    print()

# =============================================================================
# MAIN EXECUTION AND DEMO
# =============================================================================

def main():
    """
    Main function to demonstrate all queries with error handling and sample outputs.
    """
    print("🚀 MongoDB Employee Database Query System")
    print("=" * 80)
    
    # Initialize query manager
    query_manager = EmployeeQueryManager()
    
    # Execute all queries
    all_results = query_manager.run_all_queries()
    
    if not all_results.get("query_1_high_performers", {}).get("success", False):
        print("❌ Database connection failed. Please ensure MongoDB is running and the employee data is inserted.")
        print("\n📋 To insert sample data, run:")
        print("   # First run the employee data generator from the previous artifact")
        print("   # Then insert data with:")
        print("   client = MongoClient('mongodb://localhost:27017/')")
        print("   db = client['company_db']")
        print("   collection = db['employees']")
        print("   collection.insert_many(employee_db)")
        return
    
    # Display results for each query
    queries = [
        ("High Performers (Rating ≥ 4.0 & Salary > $80K)", "query_1_high_performers"),
        ("Experienced Mid-Earners (5-10 years, $70K-$120K)", "query_2_experienced_mid_earners"),
        ("Salary Outliers (Outside $60K-$100K range)", "query_3_salary_outliers"),
        ("Recent High Performers (≤2 years, Rating > 3.5)", "query_4_recent_high_performers")
    ]
    
    for query_name, query_key in queries:
        display_query_results(all_results[query_key], query_name)
    
    # Overall summary
    print("=" * 80)
    print("📊 OVERALL SUMMARY")
    print("=" * 80)
    summary = all_results["summary"]
    print(f"✅ Successfully executed {summary['successful_queries']}/4 queries")
    print(f"📈 Total employees found across all queries: {summary['total_employees_found']}")
    print(f"⏰ Execution completed at: {summary['timestamp']}")
    
    print("\n🎯 KEY INSIGHTS:")
    for query_name, query_key in queries:
        result = all_results[query_key]
        if result.get("success") and result["total_count"] > 0:
            print(f"   • {query_name}: {result['total_count']} employees")

if __name__ == "__main__":
    main()

🚀 MongoDB Employee Database Query System


INFO:__main__:Successfully connected to MongoDB: company_db.employees
INFO:__main__:Executing all employee queries...
INFO:__main__:Executing high performers query: rating >= 4.0, salary > 80000
INFO:__main__:Executing experienced mid-earners query: 5-10 years, $70000-$120000
INFO:__main__:Executing salary outliers query: excluding $60000-$100000 range
INFO:__main__:Executing recent high performers query: hired since 2023-08-14, rating > 3.5
ERROR:__main__:Database operation failed in recent_high_performers query: Invalid $project :: caused by :: Unrecognized expression '$now', full error: {'ok': 0.0, 'errmsg': "Invalid $project :: caused by :: Unrecognized expression '$now'", 'code': 168, 'codeName': 'InvalidPipelineOperator'}
INFO:__main__:MongoDB connection closed


QUERY RESULTS: High Performers (Rating ≥ 4.0 & Salary > $80K)
✅ Query successful! Found 0 results

🔍 MongoDB Query:
   {'$and': [{'performance_rating': {'$gte': 4.0}}, {'salary': {'$gt': 80000}}]}
   Projection: {'_id': 0, 'first_name': 1, 'last_name': 1, 'department': 1, 'salary': 1, 'performance_rating': 1}

QUERY RESULTS: Experienced Mid-Earners (5-10 years, $70K-$120K)
✅ Query successful! Found 0 results

🔍 MongoDB Query:
   {'$and': [{'years_experience': {'$gte': 5, '$lte': 10}}, {'salary': {'$gte': 70000, '$lte': 120000}}]}
   Projection: {'_id': 0, 'first_name': 1, 'last_name': 1, 'email': 1, 'department': 1}

QUERY RESULTS: Salary Outliers (Outside $60K-$100K range)
✅ Query successful! Found 0 results

🔍 Aggregation Pipeline:
   Stage 1: {'$match': {'$or': [{'salary': {'$lt': 60000}}, {'salary': {'$gt': 100000}}]}}
   Stage 2: {'$project': {'_id': 0, 'full_name': {'$concat': ['$first_name', ' ', '$last_name']}, 'salary': 1, 'years_experience': 1}}
   Stage 3: {'$sort': {'salary

In [36]:
import pandas as pd
import json
import os

# Check if file exists before attempting to open it
file_path = 'employees.json'
if not os.path.exists(file_path):
    # Create a sample JSON data structure if file doesn't exist
    sample_data = [
        {"id": 1, "name": "John Doe", "department": "Engineering", "salary": 75000},
        {"id": 2, "name": "Jane Smith", "department": "Marketing", "salary": 65000}
    ]
    
    # Write sample data to file
    with open(file_path, 'w') as f:
        json.dump(sample_data, f)
    print(f"Created sample file: {file_path}")

# Load JSON data
with open(file_path, 'r') as f:
    data = json.load(f)

# Convert to DataFrame and save as Excel
df = pd.json_normalize(data)
df.to_excel('employees.xlsx', index=False)
print("Excel file created successfully.")

Created sample file: employees.json
Excel file created successfully.
